# 1.4 Propagation

In the tutorial of spikingjelly ([https://spikingjelly.readthedocs.io/zh_CN/latest/activation_based_en/basic_concept.html](https://spikingjelly.readthedocs.io/zh_CN/latest/activation_based_en/basic_concept.html), they provide the concept of single timestep and multiple time-step.

However, since we always use mutiple timesteps, I will focus on using them in this.

Note: the downside of mutipletimesteps is large memory footprint and you may want to switch to single if needed.

The outputs of single and multiple timesteps are IDENTICAL.


In [1]:
import torch
import torch.nn as nn
from spikingjelly.activation_based import neuron, functional, layer
T = 4
N = 2
C = 8
x_seq = torch.rand([T, N, C]) * 256.

net = nn.Sequential(
    layer.Linear(C, 4),
    neuron.IFNode(),
    layer.Linear(4, 2),
    neuron.IFNode()
)

print(f'net={net}')


functional.set_step_mode(net, step_mode='m')
with torch.no_grad():
    y_seq_layer_by_layer = x_seq
    #for i in range(net.__len__()):
    #    y_seq_layer_by_layer = net[i](y_seq_layer_by_layer)
    y_seq_layer_by_layer = net(y_seq_layer_by_layer)
    

    print(f'y_seq_layer_by_layer=\n{y_seq_layer_by_layer}')


net=Sequential(
  (0): Linear(in_features=8, out_features=4, bias=True)
  (1): IFNode(
    v_threshold=1.0, v_reset=0.0, detach_reset=False, step_mode=s, backend=torch
    (surrogate_function): Sigmoid(alpha=4.0, spiking=True)
  )
  (2): Linear(in_features=4, out_features=2, bias=True)
  (3): IFNode(
    v_threshold=1.0, v_reset=0.0, detach_reset=False, step_mode=s, backend=torch
    (surrogate_function): Sigmoid(alpha=4.0, spiking=True)
  )
)
y_seq_layer_by_layer=
tensor([[[1., 0.],
         [1., 0.]],

        [[1., 0.],
         [1., 0.]],

        [[1., 0.],
         [1., 0.]],

        [[1., 0.],
         [1., 0.]]])


Same code with step by step 

In [5]:
functional.set_step_mode(net, step_mode='s')
print(f'net={net}')

with torch.no_grad():
    y_seq_step_by_step = []
    for t in range(T):
        x = x_seq[t]
        y = net(x)
        y_seq_step_by_step.append(y.unsqueeze(0))
    y_seq_step_by_step = torch.cat(y_seq_step_by_step, 0)

    print(f'y_seq_step_by_step=\n{y_seq_step_by_step}')


net=Sequential(
  (0): Linear(in_features=8, out_features=4, bias=True)
  (1): IFNode(
    v_threshold=1.0, v_reset=0.0, detach_reset=False, step_mode=s, backend=torch
    (surrogate_function): Sigmoid(alpha=4.0, spiking=True)
  )
  (2): Linear(in_features=4, out_features=2, bias=True)
  (3): IFNode(
    v_threshold=1.0, v_reset=0.0, detach_reset=False, step_mode=s, backend=torch
    (surrogate_function): Sigmoid(alpha=4.0, spiking=True)
  )
)
y_seq_step_by_step=
tensor([[[1., 0.],
         [1., 0.]],

        [[1., 0.],
         [1., 0.]],

        [[1., 0.],
         [1., 0.]],

        [[1., 0.],
         [1., 0.]]])


Let's compare two results

In [6]:
max_error = (y_seq_layer_by_layer - y_seq_step_by_step).abs().max()
print(f'max_error={max_error}')

max_error=0.0
